In [1]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [2]:
boston = load_boston()
x=boston.data #independent variables
y=boston.target #target variable

In [3]:
print(len(x))
print(len(y))

506
506


In [4]:
x

array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
        4.9800e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
        9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
        4.0300e+00],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        5.6400e+00],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
        6.4800e+00],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        7.8800e+00]])

In [5]:
#Step-1
#creating 30 samples from the whole boston data points.
col=np.arange(13)
x_num=np.arange(len(x))
y_num=np.arange(len(y))
col_set=[]
X_train=[]
X_test=[]
Y_train=[]
Y_test=[]
col_train_set=[]
for i in range(30):
    #randomly choosing different feature for each sample
    col_train, col_ignore= train_test_split(col, test_size=0.50,random_state=2*i)
    col_train=np.sort(col_train)
    col_train_set.append(col_train)
    #X=x[:,col_train]
    X_tr0, X_te0, y_tr0, y_te0 = train_test_split(x_num, y_num, test_size=0.40,random_state=3*i) #60% of x data
    X_tr1, X_te1, y_tr1, y_te1 = train_test_split(X_tr0, y_tr0, test_size=0.33,random_state=2*i) #taking 40%of data from x_train(60% of x)
    #addiing up the 60% and 40% values
    X_tr_final=np.hstack((X_tr0,X_tr1))
    y_tr_final=np.hstack((y_tr0,y_tr1))
    #Corpus of 30 samples
    X_train.append(X_tr_final)
    X_test.append(X_te0)
    Y_train.append(y_tr_final)
    Y_test.append(y_te0)

In [21]:
#Step-2
from sklearn.tree import DecisionTreeRegressor
clf=DecisionTreeRegressor()
y_train_pred=[]
y_test_pred=[]
y_pred=[]
y_pred_test=[]
#y_pred corpus
for i in range(30):
    y_total_pred=[]
    clf.fit(x[:,col_train_set[i]][X_train[i]], y[Y_train[i]])
    y_pred.append(clf.predict(x[:,col_train_set[i]]))
#computing Average y_pre value    
y_sum=y_pred[0]
for j in range(len(y_pred)):
    y_sum=+np.add(y_sum,y_pred[j])
y_pred_avg=np.true_divide(y_sum, 30)
#computing MSE:
y_mse=np.subtract(y,y_pred_avg)#y-y_pred
y_mse_sqr=np.square(y_mse)#squaring the diff
y_mse_sum=0
for i in range(len(y_mse_sqr)):
    y_mse_sum=y_mse_sum+y_mse_sqr[i]       
y_mse_final=y_mse_sum/len(y_mse_sqr)
print(y_mse_final)

2.8399552383740767


In [29]:
#Step-3:
y_pred_oob=[]
countp=[]

X_trains=np.asarray(X_train)
for i in range(len(x)):
    count=0
    sum1=0
    for j in range(30): 
        if i in X_test[j]:
            #print(i,j)
            count=count+1
            #print(count)
            clf.fit(x[:,col_train_set[j]][X_train[j]], y[Y_train[j]])
            sum1=sum1+clf.predict((x[:,col_train_set[0]][0].reshape(1,-1)))
    y_pred_oob.append(sum1/count) 
    
#computing OOB SCORE:
y_oob=np.subtract(y,y_pred_oob)#y-y_pred_oob
y_oob_sqr=np.square(y_oob)#squaring the diff
y_oob_sum=0
for i in range(len(y_oob_sqr)):
    y_oob_sum=y_oob_sum+y_oob_sqr[i] 
y_oob_final=y_oob_sum/(len(y_oob_sqr))
print((y_oob_final))    

[  5.0700911   17.54338384  89.62999431  67.13636121 119.78418634
  14.01322616   9.35701694   6.00875465  82.29913092  45.34583818
 111.24493888  45.34583818  16.79366331  28.10003021  54.93388189
  33.34863287   8.39757588  65.50192561  30.13947128  54.93388189
 142.32102631  36.73779446 107.12549782 121.89354153  99.12661569
 135.33186472  80.52941039 115.44437994  52.09444083  22.46170703
 164.36851109 121.89354153 151.91990843 154.36962896 144.69074684
  45.34583818  32.25891234  22.46170703   3.60204739  32.28909501
  93.39055324   4.5573573    3.1237242    3.60204739  20.74226597
  40.30695605  32.25891234  80.52941039 124.08326206  39.09723552
  35.58807393  27.11030968   3.2728858    7.10841429  45.34583818
 103.14195059   3.60204739  41.57133076   7.51813482  36.73779446
  47.98527924  91.44773357  13.34506066   3.2728858   60.89524333
   6.71869376  39.09723552  14.66450172  67.09164614  23.35142756
   4.55065004  16.79366331   9.86673747   7.10841429   4.80037057
  19.10282